<a href="https://colab.research.google.com/github/FlintyTub49/Sem_V-Machine-Learning/blob/master/Exercise%2010/Exercise%2010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing The Packages

In [1]:
import pandas as pd
import numpy as np
import re
from math import sqrt

!pip install nltk
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.multioutput import RegressorChain

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.svm import LinearSVR

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

## Donwloading and Reading the Dataset

In [2]:
!wget -O train.csv.zip https://github.com/pranavn91/blockchain/blob/master/train.csv.zip?raw=true
!wget -O test.csv.zip https://github.com/pranavn91/blockchain/blob/master/test.csv.zip?raw=true
!unzip train.csv.zip
!unzip test.csv.zip

--2020-10-20 05:17:07--  https://github.com/pranavn91/blockchain/blob/master/train.csv.zip?raw=true
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/pranavn91/blockchain/raw/master/train.csv.zip [following]
--2020-10-20 05:17:07--  https://github.com/pranavn91/blockchain/raw/master/train.csv.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pranavn91/blockchain/master/train.csv.zip [following]
--2020-10-20 05:17:07--  https://raw.githubusercontent.com/pranavn91/blockchain/master/train.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response.

In [3]:
train = pd.read_csv("train.csv", error_bad_lines=False)
test = pd.read_csv("test.csv", error_bad_lines=False)
print(train.shape)
print(test.shape)

(77946, 28)
(42157, 4)


## Preprocessing

### Keeping Relevant Columns

In [4]:
target_cols = train.columns.difference(test.columns)
target = train[target_cols]
train = train.drop(target_cols, axis = 1)

### Cleaning Location and State Columns

In [5]:
for data in (train,test):
    missing_loc_idx = data.index[data.location.isnull()].tolist()
    missing_loc_dict = {}
    for i in range(len(data)):
        if i in missing_loc_idx:
            missing_loc_dict[i] = 1
        else:
            missing_loc_dict[i] = 0
    data['missing_loc'] = data.index.map(missing_loc_dict)

for data in (train,test):
    missing_state_idx = data.index[data.state.isnull()].tolist()
    missing_state_dict = {}
    for i in range(len(data)):
        if i in missing_state_idx:
            missing_state_dict[i] = 1
        else:
            missing_state_dict[i] = 0
    data['missing_state']= data.index.map(missing_state_dict)

### Replacing NA Values

In [6]:
for data in (train,test):
    data['location'] = data['location'].replace(np.nan, '', regex=True)
    data['state'] = data['state'].replace(np.nan, '', regex=True)
    data['full_text'] = data['tweet']+' '+data['state']+' '+data['location']

In [7]:
states = {"AL":"Alabama", "AK":"Alaska", "AZ":"Arizona", "AR":"Arkansas", "CA":"California", "CO":"Colorado", "CT":"Connecticut", 
          "DE":"Delaware", "FL":"Florida", "GA":"Georgia", "HI":"Hawaii", "ID":"Idaho", "IL":"Illinois", "IN":"Indiana", "IA":"Iowa",
          "KS":"Kansas", "KY":"Kentucky", "LA":"Louisiana", "ME":"Maine", "MD":"Maryland", "MA":"Massachusetts", "MI":"Michigan", "MN":"Minnesota",
          "MS":"Mississippi", "MO":"Missouri", "MT":"Montana", "NE":"Nebraska", "NV":"Nevada", "NH":"New Hampshire", "NJ":"New Jersey", 
          "NM":"New Mexico", "NY":"New York", "NC":"North Carolina", "ND":"North Dakota", "OH":"Ohio", "OK":"Oklahoma", "OR":"Oregon", 
          "PA":"Pennsylvania", "RI":"Rhode Island", "SC":"South Carolina", "SD":"South Dakota", "TN":"Tennessee", "TX":"Texas", "UT":"Utah",
          "VT":"Vermont", "VA":"Virginia", "WA":"Washington", "WV":"West Virginia", "WI":"Wisconsin", "WY":"Wyoming", "NYC":"New York"}

## NLP

### Splitting The Tweets

In [8]:
def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|)|\|/||:]',r' ',cleaned)      
    return cleaned

def acronym(s, dct):
  return ' '.join([dct.get(i, i) for i in s.split()])

for data in (train,test):
    full_text_dict = {}
    for i,row in data.iterrows():
            full_text_dict[i] = acronym(cleanpunc(row['full_text']),states)
            #print(i)       
    data['full_txt2'] = data.index.map(full_text_dict)

for data in (train,test):
    data['num_word'] = data['tweet'].apply(lambda x : len(x.split()))
    data['num_char'] = data['tweet'].apply(lambda x : len(x))
    data['avg_word_length']= data['num_char']+1/data['num_word']

### Handling Mentions and Links

In [9]:
for data in (train,test):
    linkword = "{link}"
    link_dict = {}
    for idx in range(len(data)):
        text = data.loc[idx,'full_text']
        if linkword in text.split():
            link_dict[idx]=1
        else:
            link_dict[idx]=0
    data['link_word'] = data.index.map(link_dict)

for data in (train,test):
    mention= "@mention"
    mention_dict = {}
    for idx in range(len(data)):
        text = data.loc[idx,'full_text']
        if mention in cleanpunc(text).split():
            mention_dict[idx]=1
        else:
            mention_dict[idx]=0
    data['mention_word'] = data.index.map(mention_dict)

### Cleaning Punctuation and Links In Sentence

In [10]:
nltk.download('stopwords')
ps = PorterStemmer()
stopwords = set(stopwords.words('english'))

def cleanhtml (sentence):
    cleantext = re.sub(r'http\S+',r'',sentence)
    return cleantext

def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|)|\|/|:]',r' ',cleaned)      
    return cleaned

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
for data in (train,test):
    str1=' '
    final_string=[]
    s=''
    for sent in data['full_text']:
        filter_sent = []
        rem_html = cleanhtml(sent)
        rem_punc = cleanpunc (rem_html)
        for w in rem_punc.split():
            if ((w.isalpha())):
                if (w.lower() not in stopwords):
                    s=(ps.stem(w.lower())).encode('utf8')
                    s=(w.lower()).encode('utf8')
                    filter_sent.append(s)
                else:
                    continue
            else:
                continue
        str1 = b" ".join(filter_sent)
        final_string.append(str1)
    data['clean_text'] = np.array(final_string)

### Vectorizing and Truncating

In [12]:
vectorizer = TfidfVectorizer(encoding='utf-8', ngram_range=(1, 3), max_df=0.75, min_df=5) #max_features=12500

train_vec = vectorizer.fit_transform(train.clean_text)
test_vec = vectorizer.transform(test.clean_text)

svd = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)
train_svd = svd.fit_transform(train_vec)
test_svd = svd.transform(test_vec)

In [13]:
col = ['svd1','svd2']
svd_train =  pd.DataFrame(train_svd, columns=col)
svd_test = pd.DataFrame(test_svd, columns=col)

In [14]:
ohe = OneHotEncoder()
vectorizer2 = TfidfVectorizer(encoding='utf-8', ngram_range=(1, 3), max_df=0.75, min_df=5,max_features=50000)

X = sp.sparse.hstack((vectorizer2.fit_transform(train.clean_text),normalize(train[['num_word', 'num_char', 'avg_word_length']].values),train[['missing_loc', 'missing_state','link_word', 'mention_word']],svd_train [['svd1','svd2']]),format='csr')

X_columns=vectorizer.get_feature_names()+train[['num_word', 'num_char', 'avg_word_length','missing_loc', 'missing_state','link_word', 'mention_word']].columns.tolist() + svd_train [['svd1','svd2']].columns.tolist()
print(X.shape)
test_sp = sp.sparse.hstack((vectorizer2.transform(test.clean_text),normalize(test[['num_word', 'num_char', 'avg_word_length']].values),test[['missing_loc', 'missing_state','link_word', 'mention_word']],svd_test [['svd1','svd2']]),format='csr')

(77946, 32439)


In [15]:
test_columns=vectorizer.get_feature_names()+test[['num_word', 'num_char', 'avg_word_length','missing_loc', 'missing_state','link_word', 'mention_word']].columns.tolist() + svd_test [['svd1','svd2']].columns.tolist()

print(test_sp.shape)

(42157, 32439)


## Buidling a Model

### Splitting Data into train and test

In [16]:
# Splitting Into train and test
x, x_test, y, y_test = train_test_split(X,target,test_size=0.2,train_size=0.8, random_state = 0)

cols = target.columns
params = {'n_estimators': 500, 'max_depth': 3, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
dict_preds = {}

clf = Ridge(random_state=25)
for col in cols:
    print('start',col)
    clf.fit(x,y[col])
    rmse = sqrt(mean_squared_error(y_test[col], clf.predict(x_test)))
    print("RMSE: %.4f" % rmse)
    dict_preds[col]=clf.predict(test_sp)

start k1
RMSE: 0.0691
start k10
RMSE: 0.1242
start k11
RMSE: 0.0856
start k12
RMSE: 0.1363
start k13
RMSE: 0.1511
start k14
RMSE: 0.0504
start k15
RMSE: 0.0842
start k2
RMSE: 0.1420
start k3
RMSE: 0.0471
start k4
RMSE: 0.1569
start k5
RMSE: 0.0697
start k6
RMSE: 0.0216
start k7
RMSE: 0.2419
start k8
RMSE: 0.0324
start k9
RMSE: 0.1461
start s1
RMSE: 0.1145
start s2
RMSE: 0.2436
start s3
RMSE: 0.2168
start s4
RMSE: 0.2297
start s5
RMSE: 0.2079
start w1
RMSE: 0.2833
start w2
RMSE: 0.1760
start w3
RMSE: 0.2315
start w4
RMSE: 0.1546


### Checking Accuracy

In [17]:
regressor = RegressorChain(Ridge(random_state=25))
regressor.fit(x,y)
predictions = regressor.predict(x_test)
sqrt(mean_squared_error(y_test,predictions))

0.1601981067951308